# Chapter 10: Tools box

📈Join our community: https://discord.gg/wXjNPAc5BH

📚Read our book: https://www.amazon.com/gp/product/B09HG18CYL 

🖥️Quantreo's YouTube channel: https://www.youtube.com/channel/UCp7jckfiEglNf_Gj62VR0pw

In [7]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np

class MT5:
    
    max_price={}
    min_price={}
    summary=None
    
    def get_ticks(symbol, number_of_data = 10000):
        # Compute now date
        from_date = datetime.now()

        # Extract n Ticks before now
        ticks = mt5.copy_ticks_from(symbol, from_date, number_of_data,  mt5.COPY_TICKS_ALL)

        # Transform Tuple into a DataFrame
        df_ticks = pd.DataFrame(ticks)

        # Convert number format of the date into date format
        df_ticks["time"] = pd.to_datetime(df_ticks["time"], unit="s")

        df_ticks = df_ticks.set_index("time")

        return df_ticks
    
    
    def get_rates(symbol, number_of_data = 10000, timeframe=mt5.TIMEFRAME_D1):
        # Compute now date
        from_date = datetime.now()

        # Extract n Ticks before now
        rates = mt5.copy_rates_from(symbol, timeframe, from_date, number_of_data)


        # Transform Tuple into a DataFrame
        df_rates = pd.DataFrame(rates)

        # Convert number format of the date into date format
        df_rates["time"] = pd.to_datetime(df_rates["time"], unit="s")

        df_rates = df_rates.set_index("time")

        return df_rates
    
    def risk_reward_threshold(symbol, buy=True, risk=0.01, reward=0.02):
    
        # Extract the leverage
        leverage = mt5.account_info().leverage

        # Compute the price
        price = mt5.symbol_info(symbol).ask
        
        # Extract the number of decimals
        nb_decimal = str(price)[::-1].find(".")


        # Compute the variations in percentage
        var_down = risk/leverage
        var_up = reward/leverage


        # Find the TP and SL threshold in absolute price
        if buy:
            price = mt5.symbol_info(symbol).ask
            
            # Compute the variations in absolute price
            price_var_down = var_down*price
            price_var_up = var_up * price
            
            tp = np.round(price + price_var_up, nb_decimal)
            sl = np.round(price - price_var_down, nb_decimal)
        
        else:
            
            price = mt5.symbol_info(symbol).bid
            
            # Compute the variations in absolute price
            price_var_down = var_down*price
            price_var_up = var_up * price
            
            tp = np.round(price - price_var_up, nb_decimal)
            sl = np.round(price + price_var_down, nb_decimal)


        return tp, sl
    
    def find_filling_mode(symbol):
    
        for i in range(2):
            request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": mt5.symbol_info(symbol).volume_min,
            "type": mt5.ORDER_TYPE_BUY,
            "price": mt5.symbol_info_tick(symbol).ask,
            "type_filling": i,
            "type_time": mt5.ORDER_TIME_GTC}

            result = mt5.order_check(request)
            
            if result.comment == "Done":
                break

        return i
        
        
    def send_order(symbol, lot, buy, sell, id_position=None, pct_tp=0.02, pct_sl=0.01, comment=" No specific comment", magic=0):
    
        # Initialize the bound between MT5 and Python
        mt5.initialize()

        # Extract filling_mode
        filling_type = MT5.find_filling_mode(symbol)


        """ OPEN A TRADE """
        if buy and id_position==None:
            tp, sl = MT5.risk_reward_threshold(symbol, buy=True, risk=pct_sl, reward=pct_tp)
            
            request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot,
            "type": mt5.ORDER_TYPE_BUY,
            "price": mt5.symbol_info_tick(symbol).ask,
            "deviation": 10,
            "tp": tp,
            "sl": sl, 
            "magic": magic,
            "comment": comment,
            "type_filling": filling_type,
            "type_time": mt5.ORDER_TIME_GTC}

            result = mt5.order_send(request)
            return result

        if sell and id_position==None:
            tp, sl = MT5.risk_reward_threshold(symbol, buy=False, risk=pct_sl, reward=pct_tp)
            request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot,
            "type": mt5.ORDER_TYPE_SELL,
            "price": mt5.symbol_info_tick(symbol).bid,
            "deviation": 10,
            "tp": tp,
            "sl": sl, 
            "magic": magic,
            "comment": comment,
            "type_filling": filling_type,
            "type_time": mt5.ORDER_TIME_GTC}

            result = mt5.order_send(request)
            return result


        """ CLOSE A TRADE """
        if buy and id_position!=None:
            request = {
            "position": id_position,
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot,
            "type": mt5.ORDER_TYPE_SELL,
            "price": mt5.symbol_info_tick(symbol).bid,
            "deviation": 10,
            "magic": magic,
            "comment": comment,
            "type_filling": filling_type,
            "type_time": mt5.ORDER_TIME_GTC}

            result = mt5.order_send(request)
            return result

        if sell and id_position!=None:
            request = {
            "position": id_position,
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot,
            "type": mt5.ORDER_TYPE_BUY,
            "price": mt5.symbol_info_tick(symbol).ask,
            "deviation": 10,
            "magic": magic,
            "comment": comment,
            "type_filling": filling_type,
            "type_time": mt5.ORDER_TIME_GTC}

            result = mt5.order_send(request)
            return result

    def resume():
        """ Return the current positions. Position=0 --> Buy """    
        
        # Define the name of the columns that we will create
        colonnes = ["ticket", "position", "symbol", "volume", "magic", "profit", "price", "tp", "sl","trade_size"]

        # Go take the current open trades
        liste = mt5.positions_get()

        # Create a empty dataframe
        summary = pd.DataFrame()

        # Loop to add each row in dataframe
        for element in liste:
            element_pandas = pd.DataFrame([element.ticket, element.type, element.symbol, element.volume, element.magic,
                                           element.profit, element.price_open, element.tp,
                                           element.sl, mt5.symbol_info(element.symbol).trade_contract_size],
                                          index=colonnes).transpose()
            summary = pd.concat((summary, element_pandas), axis=0)

        try:
            summary["profit %"] = summary.profit / (summary.price * summary.trade_size * summary.volume)
            summary = summary.reset_index(drop=True)
        except:
            pass
        return summary
    
    
    def trailing_stop_loss():

        # Extract the current open positions
        MT5.summary = MT5.resume()

        # Verification: Is there any open position?
        if MT5.summary.shape[0] >0:
            for i in range(MT5.summary.shape[0]):

                # Extract information
                row = MT5.summary.iloc[i]
                symbol = row["symbol"]





                """ CASE 1: Change dynamicly the stop loss for a BUY ORDER """
                # Trailing stop loss for a buy order
                if row["position"] == 0:

                    if symbol not in MT5.max_price.keys():
                        MT5.max_price[symbol]=row["price"]

                    # Extract current price 
                    current_price = (mt5.symbol_info(symbol).ask + mt5.symbol_info(symbol).bid ) / 2

                    #Compute distance between current price an max price
                    from_sl_to_curent_price = current_price - row["sl"]
                    from_sl_to_max_price = MT5.max_price[symbol] - row["sl"]


                    # If current price is greater than preivous max price --> new max price
                    if current_price > MT5.max_price[symbol]:
                        MT5.max_price[symbol] = current_price


                    # Find the difference between the current minus max 
                    if from_sl_to_curent_price > from_sl_to_max_price:
                        difference = from_sl_to_curent_price - from_sl_to_max_price

                        # Set filling mode
                        filling_type = mt5.symbol_info(symbol).filling_mode

                        # Set the point
                        point = mt5.symbol_info(symbol).point

                        # Change the sl
                        request = {
                        "action": mt5.TRADE_ACTION_SLTP,
                        "symbol": symbol,
                        "position": row["ticket"],
                        "volume": row["volume"],
                        "type": mt5.ORDER_TYPE_BUY,
                        "price": row["price"],
                        "sl": row["sl"] + difference,
                        "type_filling": filling_type,
                        "type_time": mt5.ORDER_TIME_GTC,
                        }

                        information = mt5.order_send(request)
                        print(information)


                """ CASE 2: Change dynamicly the stop loss for a SELL ORDER """
                # Trailing stop loss for a sell order
                if row["position"] == 1:

                    if symbol not in MT5.min_price.keys():
                        MT5.min_price[symbol]=row["price"]

                    # Extract current price 
                    current_price = (mt5.symbol_info(symbol).ask + mt5.symbol_info(symbol).bid ) / 2



                    #Compute distance between current price an max price
                    from_sl_to_curent_price = row["sl"] - current_price
                    from_sl_to_min_price = row["sl"] - MT5.min_price[symbol]

                     # If current price is greater than preivous max price --> new max price
                    if current_price < MT5.min_price[symbol]:
                        MT5.min_price[symbol] = current_price


                    # Find the difference between the current minus max 
                    if from_sl_to_curent_price > from_sl_to_min_price:
                        difference = from_sl_to_curent_price - from_sl_to_min_price 

                        # Set filling mode
                        filling_type = mt5.symbol_info(symbol).filling_mode

                        # Set the point
                        point = mt5.symbol_info(symbol).point

                        # Change the sl
                        request = {
                        "action": mt5.TRADE_ACTION_SLTP,
                        "symbol": symbol,
                        "position": row["ticket"],
                        "volume": row["volume"],
                        "type": mt5.ORDER_TYPE_SELL,
                        "price": row["price"],
                        "sl": row["sl"] - difference,
                        "type_filling": filling_type,
                        "type_time": mt5.ORDER_TIME_GTC,
                        }


                        information = mt5.order_send(request)
                        print(information)
                        
    def verif_tsl():

        #print("MAX", MT5.max_price)

        #print("MIN", MT5.min_price)

        if len(MT5.summary)>0:
            buy_open_positions = MT5.summary.loc[MT5.summary["position"]==0]["symbol"]
            sell_open_positions = MT5.summary.loc[MT5.summary["position"]==0]["symbol"]
        else:
            buy_open_positions = []
            sell_open_positions = []

        """ IF YOU CLOSE ONE OF YOUR POSITION YOU NEED TO DELETE THE PRICE IN THE MAX AND MIN PRICES DICTIONNARIES"""
        if len(MT5.max_price) != len(buy_open_positions) and len(buy_open_positions) >0:
            symbol_to_delete = []

            for symbol in MT5.max_price.keys():

                if symbol not in list(buy_open_positions):
                    symbol_to_delete.append(symbol)

            for symbol in symbol_to_delete:
                del MT5.max_price[symbol]

        if len(MT5.min_price) != len(sell_open_positions) and len(sell_open_positions) >0:
            symbol_to_delete = []

            for symbol in MT5.min_price.keys():

                if symbol not in list(sell_open_positions):
                    symbol_to_delete.append(symbol)

            for symbol in symbol_to_delete:
                del MT5.min_price[symbol]

        if len(buy_open_positions) == 0:
            MT5.max_price={}

        if len(sell_open_positions) == 0:
            MT5.min_price={}

In [5]:
MT5.send_order("BTCUSD", 0.01, True, False, pct_sl=0.10)

OrderSendResult(retcode=10009, deal=3327060, order=5404528, volume=0.01, price=47457.0, bid=0.0, ask=0.0, comment='Request executed', request_id=36, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='BTCUSD', volume=0.01, price=47457.0, stoplimit=0.0, sl=47298.8, tp=47488.6, deviation=10, type=0, type_filling=1, type_time=0, expiration=0, comment=' No specific comment', position=0, position_by=0))

In [8]:
import time 


# Infinite loop
while True:
    MT5.trailing_stop_loss()

    
    MT5.verif_tsl()
    
    
    time.sleep(1)

OrderSendResult(retcode=10009, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=37, retcode_external=0, request=TradeRequest(action=6, magic=0, order=0, symbol='EURUSD', volume=1.0, price=1.09835, stoplimit=0.0, sl=1.0895700000000001, tp=0.0, deviation=0, type=0, type_filling=2, type_time=0, expiration=0, comment='', position=5395710, position_by=0))
OrderSendResult(retcode=10009, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=38, retcode_external=0, request=TradeRequest(action=6, magic=0, order=0, symbol='EURUSD', volume=1.0, price=1.09835, stoplimit=0.0, sl=1.0895899999999998, tp=0.0, deviation=0, type=0, type_filling=2, type_time=0, expiration=0, comment='', position=5395710, position_by=0))
OrderSendResult(retcode=10009, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=39, retcode_external=0, request=TradeRequest(action=6, magic=0, order=

KeyboardInterrupt: 